<a href="https://colab.research.google.com/github/awenroberts/QM2-Project/blob/main/FINAL2_asylum_seekers_receiving_support_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
import os
import pandas as pd
import matplotlib.pyplot as plt


df = pd.read_excel(
    "/content/Copy support-local-authority-datasets-sep-2025.xlsx",
    sheet_name="Data_Asy_2425",
     header=0,
    engine="openpyxl"
)
df.head()

,Date (as at…),Quarter,Year-Quarter,UK Region / Nation,Local Authority,LAD Code,Accommodation Type,People
0,2024-12-31,Q4,2024 Q4,Scotland,Aberdeen City,S12000033,Dispersal Accommodation,44
1,2024-12-31,Q4,2024 Q4,Scotland,Aberdeen City,S12000033,Contingency Accommodation - Hotel,308
2,2025-03-31,Q1,2025 Q1,Scotland,Aberdeen City,S12000033,Dispersal Accommodation,58
3,2025-03-31,Q1,2025 Q1,Scotland,Aberdeen City,S12000033,Contingency Accommodation - Hotel,300
4,2025-03-31,Q1,2025 Q1,Scotland,Aberdeen City,S12000033,Subsistence Only,5


##Cleaning the dataset

1. check types and change if necessary
2. rename columns
3. clean the dataset - drop rows which have wither 'NaN' or 'N/A' as a value in the lad_code column. this filters out any national level data or missing values.
4. standardise 'accomodation_type' column:

**accomodation_map** is a dictionary which acts as a lookup table:
- "Subsistence Only", "Subsistence only", and "N/A - Section 98 (pre-2023)" are all mapped to the single category "Subsistence Only".
- 'Contingency Accommodation' and 'Other Accommodation' are all grouped under "Contingency Accommodation"
a new column accomodation_type_clean is made, which takes values from the original accomodation_type column and puts it in accomodation_type_clean column according to the map (if there is a match)


In [61]:
df.head()

,Date (as at…),Quarter,Year-Quarter,UK Region / Nation,Local Authority,LAD Code,Accommodation Type,People
0,2024-12-31,Q4,2024 Q4,Scotland,Aberdeen City,S12000033,Dispersal Accommodation,44
1,2024-12-31,Q4,2024 Q4,Scotland,Aberdeen City,S12000033,Contingency Accommodation - Hotel,308
2,2025-03-31,Q1,2025 Q1,Scotland,Aberdeen City,S12000033,Dispersal Accommodation,58
3,2025-03-31,Q1,2025 Q1,Scotland,Aberdeen City,S12000033,Contingency Accommodation - Hotel,300
4,2025-03-31,Q1,2025 Q1,Scotland,Aberdeen City,S12000033,Subsistence Only,5


In [62]:
#check the data types
df.dtypes

,0
Date (as at…),datetime64[ns]
Quarter,object
Year-Quarter,object
UK Region / Nation,object
Local Authority,object
LAD Code,object
Accommodation Type,object
People,int64


In [63]:
#change the data types accordingly so we can work with the data usign pandas
df['Quarter']=df['Quarter'].astype('category')
df['Local Authority']=df['Local Authority'].astype('category')
df['UK Region / Nation']=df['UK Region / Nation'].astype('category')
df['LAD Code']=df['LAD Code'].astype('string')
df['Accommodation Type']=df['Accommodation Type'].astype('category')
df['People']=df['People'].astype('int64')

#rename the columns to make things easier later
df = df.rename(columns={
    "Date (as at…)": "date",
    "Quarter": "quarter",
    "Year-Quarter": "year_quarter",
    "Support Type": "support_type",
    "UK Region / Nation": "region",
    "Local Authority": "local_authority",
    "LAD Code": "lad_code",
    "Accommodation Type": "accommodation_type",
    "People": "people"
})

#convert the date column
df["date"] = pd.to_datetime(df["date"], errors="coerce")
# Ensure date is datetime
df["date"] = pd.to_datetime(df["date"])
#check it worked
df["date"].min(), df["date"].max()
#create proper quarterly time variable
df["year_quarter"] = df["date"].dt.to_period("Q")

In [64]:
df["date"] = pd.PeriodIndex(df["date"], freq="Q").to_timestamp()

# Sort
df = df.sort_values("date")
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2668 entries, 0 to 2667
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   date                2668 non-null   datetime64[ns]
 1   quarter             2668 non-null   category      
 2   year_quarter        2668 non-null   period[Q-DEC] 
 3   region              2668 non-null   category      
 4   local_authority     2668 non-null   category      
 5   lad_code            2668 non-null   string        
 6   accommodation_type  2668 non-null   category      
 7   people              2668 non-null   int64         
dtypes: category(4), datetime64[ns](1), int64(1), period[Q-DEC](1), string(1)
memory usage: 129.1 KB


In [65]:
#cleaning
#get rid of the national level data and null values and put it into a new dataframe
df = df[~df["lad_code"].str.startswith("N/A", na=False)].copy()
df_unclean = df[df["lad_code"].isna() | df["lad_code"].str.startswith("N/A", na=False)]
df_unclean.head()

,date,quarter,year_quarter,region,local_authority,lad_code,accommodation_type,people


In [66]:
#create a map which harmonsises the dataset to include 4 accomodation types of interest
#map with 'other' merged with contingency
accommodation_map = {
    # Subsistence-only variants
    "Subsistence Only": "Subsistence Only",
    "Subsistence only": "Subsistence Only",
    "N/A - Section 98 (pre-2023)": "Subsistence Only",

    # Initial accommodation
    "Initial Accommodation": "Initial Accommodation",

    # Dispersal (long-term, normalised)
    "Dispersal Accommodation": "Dispersal Accommodation",

    # Contingency + alternatives to hotels
    "Contingency Accommodation - Hotel": "Contingency Accommodation",
    "Contingency Accommodation - Other": "Contingency Accommodation",
    "Other Accommodation": "Contingency Accommodation",
}
df["accommodation_type_clean"] = df["accommodation_type"].map(accommodation_map)


In [67]:
df["accommodation_type_clean"].value_counts()

,count
accommodation_type_clean,
Dispersal Accommodation,1187
Subsistence Only,857
Contingency Accommodation,569
Initial Accommodation,54


In [68]:
#check how many LAs are in the dataframe in 2025 (period of interest)
df_2025 = df[(df["date"]  >= "2025-01-01") & (df["date"] <= "2025-12-31")]
df_2025.columns
df_2025["lad_code"].nunique()

345

In [69]:
#Group by the specified columns and sum the 'people' column
#df_final = df_2025.groupby([
    #'date', 'quarter', 'year_quarter', 'region', 'local_authority', 'lad_code', 'accommodation_type_clean'
#], observed=False)['people'].sum().reset_index()

#df_final

##Bring in population data

In [70]:
import pandas as pd

#import population data for local authorities
xls = pd.ExcelFile("/content/Copy support-local-authority-datasets-sep-2025.xlsx")
print("Available Excel sheets:", xls.sheet_names)


pop_df = pd.read_excel(
    "/content/Copy support-local-authority-datasets-sep-2025.xlsx",
    sheet_name="population_per_LA",
    header=0,
    engine="openpyxl"
)
pop_df.head()

Available Excel sheets: ['Cover_sheet', 'Contents', 'Notes', 'List_of_Fields', 'Asy_D11', 'Data_Asy_D11', 'Data_Asy_20242025', 'Data_Asy_2425', 'Data_Asy_D11_2020_2025', 'population_per_LA', 'la_exposure_table', 'la_exposure_table_old', '2025', 'protest_data', 'population for local authoritie']


,lad_code,name,geography,population,Isy
0,E06000047,County Durham,Unitary Authority,538011,True
1,E06000005,Darlington,Unitary Authority,112489,True
2,E06000001,Hartlepool,Unitary Authority,98180,True
3,E06000002,Middlesbrough,Unitary Authority,156161,True
4,E06000057,Northumberland,Unitary Authority,331420,True


In [71]:
#check number of LAs is correct
pop_df.columns
pop_df["lad_code"].nunique()

354

In [72]:
#merge population dataframe with asylum support dataframe by lad_code
df_merged = df.merge(
    pop_df[["lad_code", "population"]],
    on="lad_code",
    how="left"
)
df_merged = df_merged[(df_merged["date"]  >= "2025-01-01") & (df_merged["date"] <= "2025-12-31")]

#group people by accomodation_type_clean
df_merged

,date,quarter,year_quarter,region,local_authority,lad_code,accommodation_type,people,accommodation_type_clean,population
455,2025-01-01,Q1,2025Q1,North West,Lancaster,E07000121,Dispersal Accommodation,185,Dispersal Accommodation,145006
456,2025-01-01,Q1,2025Q1,Yorkshire and The Humber,Kirklees,E08000034,Subsistence Only,18,Subsistence Only,447847
457,2025-01-01,Q1,2025Q1,East Midlands,West Northamptonshire,E06000062,Contingency Accommodation - Hotel,201,Contingency Accommodation,439811
458,2025-01-01,Q1,2025Q1,South East,West Oxfordshire,E07000181,Dispersal Accommodation,4,Dispersal Accommodation,120941
459,2025-01-01,Q1,2025Q1,North West,Knowsley,E08000011,Dispersal Accommodation,316,Dispersal Accommodation,162565
...,...,...,...,...,...,...,...,...,...,...
2662,2025-07-01,Q3,2025Q3,South East,South Oxfordshire,E07000179,Dispersal Accommodation,19,Dispersal Accommodation,156470
2663,2025-07-01,Q3,2025Q3,South East,South Oxfordshire,E07000179,Subsistence Only,1,Subsistence Only,156470
2664,2025-07-01,Q3,2025Q3,East of England,Broxbourne,E07000095,Subsistence Only,20,Subsistence Only,101900
2665,2025-07-01,Q3,2025Q3,Scotland,South Lanarkshire,S12000029,Contingency Accommodation - Hotel,98,Contingency Accommodation,334030


In [73]:
# Group by the specified columns and sum the 'people' column
#df_final = df_merged.groupby([
    #'date', 'quarter', 'year_quarter', 'region', 'local_authority', 'lad_code', 'accommodation_type_clean', 'population'
#], observed=False)['people'].sum().reset_index()

#df_final

In [74]:
df_merged

,date,quarter,year_quarter,region,local_authority,lad_code,accommodation_type,people,accommodation_type_clean,population
455,2025-01-01,Q1,2025Q1,North West,Lancaster,E07000121,Dispersal Accommodation,185,Dispersal Accommodation,145006
456,2025-01-01,Q1,2025Q1,Yorkshire and The Humber,Kirklees,E08000034,Subsistence Only,18,Subsistence Only,447847
457,2025-01-01,Q1,2025Q1,East Midlands,West Northamptonshire,E06000062,Contingency Accommodation - Hotel,201,Contingency Accommodation,439811
458,2025-01-01,Q1,2025Q1,South East,West Oxfordshire,E07000181,Dispersal Accommodation,4,Dispersal Accommodation,120941
459,2025-01-01,Q1,2025Q1,North West,Knowsley,E08000011,Dispersal Accommodation,316,Dispersal Accommodation,162565
...,...,...,...,...,...,...,...,...,...,...
2662,2025-07-01,Q3,2025Q3,South East,South Oxfordshire,E07000179,Dispersal Accommodation,19,Dispersal Accommodation,156470
2663,2025-07-01,Q3,2025Q3,South East,South Oxfordshire,E07000179,Subsistence Only,1,Subsistence Only,156470
2664,2025-07-01,Q3,2025Q3,East of England,Broxbourne,E07000095,Subsistence Only,20,Subsistence Only,101900
2665,2025-07-01,Q3,2025Q3,Scotland,South Lanarkshire,S12000029,Contingency Accommodation - Hotel,98,Contingency Accommodation,334030


#Metric 1 — Contingency accommodation per 1,000 residents
This captures local visibility / pressure, which is what protests respond to.

0.5 = 1 person in contingency per 2,000 residents
2.0 = very high local exposure

In [75]:
#create a new dataframe (copy) which filters df_merged for rows where the 'accommodation_type_clean' has the value 'Contingency Accomodation'
contingency_df = df_merged[
    df_merged["accommodation_type_clean"] == "Contingency Accommodation"
].copy()
contingency_df

,date,quarter,year_quarter,region,local_authority,lad_code,accommodation_type,people,accommodation_type_clean,population
457,2025-01-01,Q1,2025Q1,East Midlands,West Northamptonshire,E06000062,Contingency Accommodation - Hotel,201,Contingency Accommodation,439811
462,2025-01-01,Q1,2025Q1,South East,West Oxfordshire,E07000181,Contingency Accommodation - Hotel,158,Contingency Accommodation,120941
463,2025-01-01,Q1,2025Q1,London,Lambeth,E09000022,Contingency Accommodation - Hotel,415,Contingency Accommodation,316920
478,2025-01-01,Q1,2025Q1,London,Islington,E09000019,Contingency Accommodation - Hotel,457,Contingency Accommodation,223024
479,2025-01-01,Q1,2025Q1,London,Islington,E09000019,Contingency Accommodation - Other,170,Contingency Accommodation,223024
...,...,...,...,...,...,...,...,...,...,...
2637,2025-07-01,Q3,2025Q3,North West,South Ribble,E07000126,Contingency Accommodation - Hotel,171,Contingency Accommodation,116113
2640,2025-07-01,Q3,2025Q3,East of England,Broxbourne,E07000095,Contingency Accommodation - Hotel,291,Contingency Accommodation,101900
2641,2025-07-01,Q3,2025Q3,North West,Manchester,E08000003,Contingency Accommodation - Hotel,1259,Contingency Accommodation,589670
2661,2025-07-01,Q3,2025Q3,East of England,Luton,E06000032,Contingency Accommodation - Other,435,Contingency Accommodation,239090


In [86]:

#divide the number of people in contingency accomodation by the population in each LA then *1000
contingency_df["contingency_per_1000"] = (
    contingency_df["people"] / contingency_df["population"]
) * 1000

#sort by lad_code
contingency_df = contingency_df.sort_values("lad_code")
contingency_df


,date,quarter,year_quarter,region,local_authority,lad_code,accommodation_type,people,accommodation_type_clean,population,contingency_per_1000
861,2025-01-01,Q1,2025Q1,North East,Stockton-on-Tees,E06000004,Contingency Accommodation - Other,19,Contingency Accommodation,206800,0.091876
1519,2025-04-01,Q2,2025Q2,North East,Stockton-on-Tees,E06000004,Contingency Accommodation - Other,18,Contingency Accommodation,206800,0.087041
2547,2025-07-01,Q3,2025Q3,North East,Stockton-on-Tees,E06000004,Contingency Accommodation - Other,31,Contingency Accommodation,206800,0.149903
583,2025-01-01,Q1,2025Q1,North West,Halton,E06000006,Contingency Accommodation - Hotel,270,Contingency Accommodation,131543,2.052561
1388,2025-04-01,Q2,2025Q2,North West,Warrington,E06000007,Contingency Accommodation - Hotel,240,Contingency Accommodation,215391,1.114253
...,...,...,...,...,...,...,...,...,...,...,...
1322,2025-04-01,Q2,2025Q2,Scotland,Perth and Kinross,S12000048,Contingency Accommodation - Hotel,192,Contingency Accommodation,154420,1.243362
740,2025-01-01,Q1,2025Q1,Scotland,Perth and Kinross,S12000048,Contingency Accommodation - Hotel,171,Contingency Accommodation,154420,1.107370
1354,2025-04-01,Q2,2025Q2,Wales,Cardiff,W06000015,Contingency Accommodation - Hotel,76,Contingency Accommodation,383919,0.197958
2330,2025-07-01,Q3,2025Q3,Wales,Cardiff,W06000015,Contingency Accommodation - Hotel,99,Contingency Accommodation,383919,0.257867


#Metric 2 — Change in contingency accommodation (Q1 → Q3 2025)
This captures local shocks, not national trends.

In [90]:
# Define the quarters to filter
quarters_to_filter = pd.PeriodIndex(["2025Q1", "2025Q2", "2025Q3"], freq="Q")

# Filter df_merged for the relevant quarters and accommodation type
temp_df = df_merged[
    (df_merged["year_quarter"].isin(quarters_to_filter)) &
    (df_merged["accommodation_type_clean"] == "Contingency Accommodation")
].copy()

# Pivot the filtered data
cont_wide = pd.pivot_table(
    temp_df,
    index="lad_code",
    columns="year_quarter",
    values="people",
    aggfunc="sum"
).reset_index()

# Rename columns for clarity
cont_wide = cont_wide.rename(columns={
    pd.Period('2025Q1', freq='Q'): '2025Q1_contingency',
    pd.Period('2025Q2', freq='Q'): '2025Q2_contingency', # Added Q2
    pd.Period('2025Q3', freq='Q'): '2025Q3_contingency'
})

# Get all unique lad_codes and their local_authority names
all_lad_codes_df = df_merged[['lad_code', 'local_authority']].drop_duplicates()

# Merge with the full list of lad_codes to ensure all are present
cont_wide = pd.merge(all_lad_codes_df, cont_wide, on='lad_code', how='left')

# Fill NaN values in contingency columns with 0
contingency_cols = ['2025Q1_contingency', '2025Q2_contingency', '2025Q3_contingency']
cont_wide[contingency_cols] = cont_wide[contingency_cols].fillna(0)

# Compute the change
cont_wide["delta_contingency_Q1_Q3"] = (
    cont_wide['2025Q3_contingency'] - cont_wide['2025Q1_contingency']
)

cont_wide

,lad_code,local_authority,2025Q1_contingency,2025Q2_contingency,2025Q3_contingency,delta_contingency_Q1_Q3
0,E07000121,Lancaster,0.0,0.0,0.0,0.0
1,E08000034,Kirklees,0.0,0.0,0.0,0.0
2,E06000062,West Northamptonshire,201.0,398.0,404.0,203.0
3,E07000181,West Oxfordshire,158.0,124.0,235.0,77.0
4,E08000011,Knowsley,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
340,E07000010,Fenland,0.0,0.0,0.0,0.0
341,S12000020,Moray,0.0,0.0,0.0,0.0
342,S12000005,Clackmannanshire,0.0,0.0,0.0,0.0
343,S12000030,Stirling,0.0,0.0,0.0,0.0


#Metric 3: share of local asylum population in contingency accomodation

tells us how dominant levels are locally, not nationally.

In [91]:
#caculate total asylum population per LA-quarter
total_asylum = (
    df
    .groupby(["lad_code", "year_quarter"], as_index=False)
    ["people"]
    .sum()
    .rename(columns={"people": "total_asylum"})
)

total_asylum

,lad_code,year_quarter,total_asylum
0,E06000001,2024Q4,462
1,E06000001,2025Q1,443
2,E06000001,2025Q2,440
3,E06000001,2025Q3,431
4,E06000002,2024Q4,669
...,...,...,...
1303,W06000023,2025Q3,10
1304,W06000024,2024Q4,15
1305,W06000024,2025Q1,11
1306,W06000024,2025Q2,18


In [92]:
#merge totals into contingency data
contingency_total = contingency_df.merge(
    total_asylum,
    on=["lad_code", "year_quarter"],
    how="left"
)

contingency_total

,date,quarter,year_quarter,region,local_authority,lad_code,accommodation_type,people,accommodation_type_clean,population,contingency_per_1000,total_asylum
0,2025-01-01,Q1,2025Q1,North East,Stockton-on-Tees,E06000004,Contingency Accommodation - Other,19,Contingency Accommodation,206800,0.091876,800
1,2025-04-01,Q2,2025Q2,North East,Stockton-on-Tees,E06000004,Contingency Accommodation - Other,18,Contingency Accommodation,206800,0.087041,779
2,2025-07-01,Q3,2025Q3,North East,Stockton-on-Tees,E06000004,Contingency Accommodation - Other,31,Contingency Accommodation,206800,0.149903,805
3,2025-01-01,Q1,2025Q1,North West,Halton,E06000006,Contingency Accommodation - Hotel,270,Contingency Accommodation,131543,2.052561,867
4,2025-04-01,Q2,2025Q2,North West,Warrington,E06000007,Contingency Accommodation - Hotel,240,Contingency Accommodation,215391,1.114253,446
...,...,...,...,...,...,...,...,...,...,...,...,...
415,2025-04-01,Q2,2025Q2,Scotland,Perth and Kinross,S12000048,Contingency Accommodation - Hotel,192,Contingency Accommodation,154420,1.243362,195
416,2025-01-01,Q1,2025Q1,Scotland,Perth and Kinross,S12000048,Contingency Accommodation - Hotel,171,Contingency Accommodation,154420,1.107370,175
417,2025-04-01,Q2,2025Q2,Wales,Cardiff,W06000015,Contingency Accommodation - Hotel,76,Contingency Accommodation,383919,0.197958,1442
418,2025-07-01,Q3,2025Q3,Wales,Cardiff,W06000015,Contingency Accommodation - Hotel,99,Contingency Accommodation,383919,0.257867,1463


In [83]:
print(f"Shape of contingency_total: {contingency_total.shape}")
print(f"Number of null values in 'total_asylum' column: {contingency_total['total_asylum'].isnull().sum()}")

Shape of contingency_total: (420, 12)
Number of null values in 'total_asylum' column: 0


In [93]:
#compute share
contingency_total["contingency_share"] = (
    contingency_total["people"] / contingency_total["total_asylum"]
)
contingency_total

,date,quarter,year_quarter,region,local_authority,lad_code,accommodation_type,people,accommodation_type_clean,population,contingency_per_1000,total_asylum,contingency_share
0,2025-01-01,Q1,2025Q1,North East,Stockton-on-Tees,E06000004,Contingency Accommodation - Other,19,Contingency Accommodation,206800,0.091876,800,0.023750
1,2025-04-01,Q2,2025Q2,North East,Stockton-on-Tees,E06000004,Contingency Accommodation - Other,18,Contingency Accommodation,206800,0.087041,779,0.023107
2,2025-07-01,Q3,2025Q3,North East,Stockton-on-Tees,E06000004,Contingency Accommodation - Other,31,Contingency Accommodation,206800,0.149903,805,0.038509
3,2025-01-01,Q1,2025Q1,North West,Halton,E06000006,Contingency Accommodation - Hotel,270,Contingency Accommodation,131543,2.052561,867,0.311419
4,2025-04-01,Q2,2025Q2,North West,Warrington,E06000007,Contingency Accommodation - Hotel,240,Contingency Accommodation,215391,1.114253,446,0.538117
...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,2025-04-01,Q2,2025Q2,Scotland,Perth and Kinross,S12000048,Contingency Accommodation - Hotel,192,Contingency Accommodation,154420,1.243362,195,0.984615
416,2025-01-01,Q1,2025Q1,Scotland,Perth and Kinross,S12000048,Contingency Accommodation - Hotel,171,Contingency Accommodation,154420,1.107370,175,0.977143
417,2025-04-01,Q2,2025Q2,Wales,Cardiff,W06000015,Contingency Accommodation - Hotel,76,Contingency Accommodation,383919,0.197958,1442,0.052705
418,2025-07-01,Q3,2025Q3,Wales,Cardiff,W06000015,Contingency Accommodation - Hotel,99,Contingency Accommodation,383919,0.257867,1463,0.067669


In [97]:
#final LA-level exposure table
la_exposure = (
    contingency_total
    .query("year_quarter == @pd.Period('2025Q3', freq='Q')") # Corrected query to use Period object
    [["lad_code", "local_authority", "population", "total_asylum", "contingency_per_1000", "contingency_share"]]
    .merge(
        cont_wide[["lad_code", "delta_contingency_Q1_Q3"]],
        on="lad_code",
        how="left"
    )
)
#round population column to a whole number
la_exposure["population"] = la_exposure["population"].round(0).astype(int)
#round the values in all columns to four decimal points
la_exposure["contingency_share"] = la_exposure["contingency_share"].round(4)
la_exposure["contingency_per_1000"] = la_exposure["contingency_per_1000"].round(4)
la_exposure["delta_contingency_Q1_Q3"] = la_exposure["contingency_share"].round(4)

#check for anny errors, null values or N/A in any column
#la_exposure.info()
#change object types to correct types for quantitative analysis
la_exposure["population"] = la_exposure["population"].astype(int)
la_exposure["total_asylum"] = la_exposure["total_asylum"].astype(int)
la_exposure["contingency_share"] = la_exposure["contingency_share"].astype(int)
la_exposure["contingency_per_1000"] = la_exposure["contingency_per_1000"].astype(int)
la_exposure["delta_contingency_Q1_Q3"] = la_exposure["contingency_share"].astype(int)

la_exposure

,lad_code,local_authority,population,total_asylum,contingency_per_1000,contingency_share,delta_contingency_Q1_Q3
0,E06000004,Stockton-on-Tees,206800,805,0,0,0
1,E06000007,Warrington,215391,469,1,0,0
2,E06000009,Blackpool,144191,676,3,0,0
3,E06000010,"Kingston upon Hull, City of",275401,777,0,0,0
4,E06000014,York,209301,393,1,0,0
...,...,...,...,...,...,...,...
131,S12000038,Renfrewshire,189170,98,0,0,0
132,S12000040,West Lothian,186440,89,0,0,0
133,S12000042,Dundee City,149880,160,0,0,0
134,S12000048,Perth and Kinross,154420,205,1,0,0


In [101]:
# Start with all local authorities
la_exposure = all_lad_codes_df.copy()

# Add population data
la_exposure = la_exposure.merge(
    pop_df[["lad_code", "population"]],
    on="lad_code",
    how="left"
)

# Add total asylum population for 2025Q3
total_asylum_q3 = total_asylum.query("year_quarter == @pd.Period('2025Q3', freq='Q')")
la_exposure = la_exposure.merge(
    total_asylum_q3[["lad_code", "total_asylum"]],
    on="lad_code",
    how="left"
)

# Add contingency_per_1000 and contingency_share for 2025Q3
contingency_metrics_q3 = contingency_total.query("year_quarter == @pd.Period('2025Q3', freq='Q')")
la_exposure = la_exposure.merge(
    contingency_metrics_q3[["lad_code", "contingency_per_1000", "contingency_share"]],
    on="lad_code",
    how="left"
)

# Add delta_contingency_Q1_Q3
la_exposure = la_exposure.merge(
    cont_wide[["lad_code", "delta_contingency_Q1_Q3"]],
    on="lad_code",
    how="left"
)

# Calculate total people in contingency accommodation across Q1, Q2, Q3
cont_wide["total_contingency_people_Q1_Q3"] = cont_wide[[
    "2025Q1_contingency", "2025Q2_contingency", "2025Q3_contingency"
]].sum(axis=1)

# Merge the new total contingency people column into la_exposure
la_exposure = la_exposure.merge(
    cont_wide[["lad_code", "total_contingency_people_Q1_Q3"]],
    on="lad_code",
    how="left"
)

# Rename existing columns
la_exposure = la_exposure.rename(columns={
    'total_asylum': 'total_asylumQ3',
    'contingency_share': 'contingency_shareQ3',
    'total_contingency_people_Q1_Q3': 'total_contingency'
})

# Calculate cumulative total asylum for 2025
total_asylum_cumulative_2025 = df_merged[
    df_merged['year_quarter'].dt.year == 2025
].groupby('lad_code')['people'].sum().reset_index()
total_asylum_cumulative_2025 = total_asylum_cumulative_2025.rename(columns={'people': 'total_asylum_2025_cumulative'})

# Merge cumulative total asylum into la_exposure
la_exposure = la_exposure.merge(
    total_asylum_cumulative_2025, on='lad_code', how='left'
)

# Fill NaN values for numeric columns that should be 0 if no data
numeric_cols_to_fill = [
    "population", "total_asylumQ3", "contingency_per_1000",
    "contingency_shareQ3", "delta_contingency_Q1_Q3", "total_contingency",
    "total_asylum_2025_cumulative"
]
la_exposure[numeric_cols_to_fill] = la_exposure[numeric_cols_to_fill].fillna(0)

# Calculate new cumulative contingency share for 2025
# Handle division by zero: if total_asylum_2025_cumulative is 0, the share should be 0
la_exposure['contingency_share_2025_cumulative'] = (
    la_exposure['total_contingency'] / la_exposure['total_asylum_2025_cumulative']
)
la_exposure['contingency_share_2025_cumulative'] = la_exposure['contingency_share_2025_cumulative'].replace([float('inf'), -float('inf')], 0).fillna(0)

# Round and convert dtypes
la_exposure["population"] = la_exposure["population"].round(0).astype(int)
la_exposure["total_asylumQ3"] = la_exposure["total_asylumQ3"].round(0).astype(int)
la_exposure["contingency_per_1000"] = la_exposure["contingency_per_1000"].round(4)
la_exposure["contingency_shareQ3"] = la_exposure["contingency_shareQ3"].round(4)
la_exposure["delta_contingency_Q1_Q3"] = la_exposure["delta_contingency_Q1_Q3"].round(4)
la_exposure["total_contingency"] = la_exposure["total_contingency"].round(0).astype(int)
la_exposure["total_asylum_2025_cumulative"] = la_exposure["total_asylum_2025_cumulative"].round(0).astype(int)
la_exposure["contingency_share_2025_cumulative"] = la_exposure["contingency_share_2025_cumulative"].round(4)

la_exposure

,lad_code,local_authority,population,total_asylumQ3,contingency_per_1000,contingency_shareQ3,delta_contingency_Q1_Q3,total_contingency,total_asylum_2025_cumulative,contingency_share_2025_cumulative
0,E07000121,Lancaster,145006,239,0.0000,0.0000,0.0,0,649,0.0000
1,E08000034,Kirklees,447847,748,0.0000,0.0000,0.0,0,2269,0.0000
2,E06000062,West Northamptonshire,439811,486,0.9186,0.8313,203.0,1003,1199,0.8365
3,E07000181,West Oxfordshire,120941,239,1.9431,0.9833,77.0,517,529,0.9773
4,E08000011,Knowsley,162565,312,0.0000,0.0000,0.0,0,954,0.0000
...,...,...,...,...,...,...,...,...,...,...
347,E07000010,Fenland,104896,2,0.0000,0.0000,0.0,0,2,0.0000
348,S12000020,Moray,95010,2,0.0000,0.0000,0.0,0,2,0.0000
349,S12000005,Clackmannanshire,52110,3,0.0000,0.0000,0.0,0,3,0.0000
350,S12000030,Stirling,94210,1,0.0000,0.0000,0.0,0,1,0.0000
